In [ ]:
# Referred to Lab tutorials and also referred to Learning Ray text book,
# Reference: "Pumperla, M., Oakes, E. & Liaw, R. 2023, Learning Ray: Flexible Distributed Python for Machine Learning, 1st edn, O'Reilly."

# Installing Ray
!pip install "ray[tune]"

# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
import ray
from ray import tune
import logging

# Configure logging for HPC process monitoring
logging.basicConfig(level=logging.INFO)
hpc_logger = logging.getLogger(__name__)

# Initialize Ray for parallel processing
try:
    ray.init(ignore_reinit_error=True, num_cpus=2)
    hpc_logger.info("Ray framework successfully activated for HPC")
except Exception as err:
    hpc_logger.error(f"Ray initialization failed: {err}")
    raise

# Loading the bodyfat dataset (fetch_and_clean_data function)
try:
    bodyfat_data_hpc = pd.read_csv('Raw_Data.csv')
    hpc_logger.info(f"Successfully loaded body fat dataset with shape: {bodyfat_data_hpc.shape}")
except Exception as err:
    hpc_logger.error(f"Failed to load body fat dataset: {err}")
    raise

# Removing duplicate entries
duplicate_count = bodyfat_data_hpc.duplicated().sum()
hpc_logger.info(f"Detected {duplicate_count} duplicate rows in the dataset")
bodyfat_data_hpc = bodyfat_data_hpc.drop_duplicates()

# Defining a function to filter outliers using IQR method
def filter_outliers_hpc(data, col_name):
    try:
        q1 = data[col_name].quantile(0.25)
        q3 = data[col_name].quantile(0.75)
        iqr = q3 - q1
        lower_limit = q1 - 1.5 * iqr
        upper_limit = q3 + 1.5 * iqr
        filtered_data = data[(data[col_name] >= lower_limit) & (data[col_name] <= upper_limit)]
        return filtered_data
    except Exception as err:
        hpc_logger.error(f"Outlier filtering failed for column {col_name}: {err}")
        raise

# Applying outlier removal to all columns
for col in bodyfat_data_hpc.columns:
    bodyfat_data_hpc = filter_outliers_hpc(bodyfat_data_hpc, col)
hpc_logger.info(f"Dataset shape after outlier filtering: {bodyfat_data_hpc.shape}")

# To verify absence of NaN or infinite values
if bodyfat_data_hpc.isna().any().any() or np.isinf(bodyfat_data_hpc.values).any():
    hpc_logger.error("Detected NaN or infinite values in the dataset")
    raise ValueError("Dataset contains invalid values (NaN or infinite)")

# Define a function for Exploratory Data Analysis (EDA) specific to HPC (explore_dataset function)
def perform_eda_hpc(data):
    try:
        # Correlation heatmap for selected features
        corr_data_hpc = data[['Density', 'Age', 'Weight', 'BodyFat']].corr()
        plt.figure(figsize=(8, 6))
        sns.heatmap(corr_data_hpc, annot=True, cmap='viridis', fmt='.2f')
        plt.title('BodyFat Feature Correlation Heatmap (HPC)')
        plt.savefig('bodyfat_corr_hpc.png')
        plt.close()
        sns.pairplot(data[['Density', 'Age', 'Weight', 'BodyFat']], diag_kind='kde')
        plt.savefig('bodyfat_pairplot_hpc.png')
        plt.close()

# Histogram
        plt.figure(figsize=(8, 6))
        sns.histplot(data['BodyFat'], kde=True, color='green')
        plt.title('BodyFat Distribution Histogram (HPC)')
        plt.savefig('bodyfat_dist_hpc.png')
        plt.close()
    except Exception as err:
        hpc_logger.error(f"EDA process failed: {err}")
        raise

# EDA
perform_eda_hpc(bodyfat_data_hpc)

# Preparing data for modeling
bodyfat_formula_hpc = 'BodyFat ~ Density + Age + Weight'
features_hpc = bodyfat_data_hpc[['Density', 'Age', 'Weight']]
target_hpc = bodyfat_data_hpc['BodyFat']

# Spliting the dataset
training_bodyfat_hpc, test_bodyfat_hpc, training_target_hpc, test_target_hpc = train_test_split(
    features_hpc, target_hpc, test_size=0.2, random_state=42)
hpc_logger.info(f"Training data shape: {training_bodyfat_hpc.shape}, Testing data shape: {test_bodyfat_hpc.shape}")

# Scale
scaler_hpc = StandardScaler()
training_bodyfat_scaled = scaler_hpc.fit_transform(training_bodyfat_hpc)
test_bodyfat_scaled = scaler_hpc.transform(test_bodyfat_hpc)

# Validate scaled data
if np.any(np.isnan(training_bodyfat_scaled)) or np.any(np.isnan(test_bodyfat_scaled)) or \
   np.any(np.isinf(training_bodyfat_scaled)) or np.any(np.isinf(test_bodyfat_scaled)):
    hpc_logger.error("Invalid values (NaN or infinite) found in scaled data")
    raise ValueError("Scaled data contains NaN or infinite values")

# Defining the training function for Ray Tune (preprocess_for_modeling function)
def train_bodyfat_nn2_hpc(config, X_train, y_train, X_test, y_test):
    try:
        bodyfat_nn2_hpc = MLPRegressor(
            hidden_layer_sizes=(config["neurons"],),
            max_iter=2000,
            random_state=42,
            solver='adam',
            learning_rate_init=0.001,
            early_stopping=True,
            validation_fraction=0.1,
            n_iter_no_change=10
        )
        bodyfat_nn2_hpc.fit(X_train, y_train)
        pred_bodyfat_nn2_hpc = bodyfat_nn2_hpc.predict(X_test)
        mse_hpc = mean_squared_error(y_test, pred_bodyfat_nn2_hpc)
        r2_hpc = r2_score(y_test, pred_bodyfat_nn2_hpc)
        tune.report({'mse': mse_hpc, 'r2': r2_hpc})
        hpc_logger.info(f"Configuration with {config['neurons']} neurons - MSE: {mse_hpc}, R²: {r2_hpc}")
    except Exception as err:
        hpc_logger.error(f"Training failed for config {config}: {err}")
        raise

train_bodyfat_with_data_hpc = tune.with_parameters(
    train_bodyfat_nn2_hpc,
    X_train=training_bodyfat_scaled,
    y_train=training_target_hpc,
    X_test=test_bodyfat_scaled,
    y_test=test_target_hpc
)

# Executing hyperparameter tuning
try:
    tune_analysis_hpc = tune.run(
        train_bodyfat_with_data_hpc,
        config={"neurons": tune.grid_search([50, 100, 150])},
        metric="mse",
        mode="min",
        verbose=3,
        resources_per_trial={"cpu": 1},
        max_concurrent_trials=1,
        raise_on_failed_trial=True
    )
    if tune_analysis_hpc.best_trial:
        optimal_neurons_hpc = tune_analysis_hpc.best_config["neurons"]
        optimal_mse_hpc = tune_analysis_hpc.best_result["mse"]
        optimal_r2_hpc = tune_analysis_hpc.best_result["r2"]
        hpc_logger.info(f"Optimal neurons found: {optimal_neurons_hpc}")
        hpc_logger.info(f"Best MSE achieved: {optimal_mse_hpc}")
        hpc_logger.info(f"Best R² score: {optimal_r2_hpc}")
    else:
        hpc_logger.warning("No successful trials completed in hyperparameter tuning")
except Exception as err:
    hpc_logger.error(f"Hyperparameter tuning failed: {err}")
    raise

# Training the final model with optimal hyperparameters (neural_network_trainer)
if tune_analysis_hpc.best_trial:
    try:
        bodyfat_nn2_hpc_final = MLPRegressor(
            hidden_layer_sizes=(optimal_neurons_hpc,),
            max_iter=2000,
            random_state=42,
            solver='adam',
            learning_rate_init=0.001,
            early_stopping=True,
            validation_fraction=0.1,
            n_iter_no_change=10)
        bodyfat_nn2_hpc_final.fit(training_bodyfat_scaled, training_target_hpc)
        pred_bodyfat_nn2_hpc_final = bodyfat_nn2_hpc_final.predict(test_bodyfat_scaled)
        final_mse_hpc = mean_squared_error(test_target_hpc, pred_bodyfat_nn2_hpc_final)
        final_r2_hpc = r2_score(test_target_hpc, pred_bodyfat_nn2_hpc_final)
        hpc_logger.info(f"Final HPC model MSE: {final_mse_hpc}")
        hpc_logger.info(f"Final HPC model R²: {final_r2_hpc}")

        # Table of actual vs predicted values
        bodyfat_results_hpc = pd.DataFrame({
            'Actual_BodyFat': test_target_hpc,
            'Predicted_BodyFat': pred_bodyfat_nn2_hpc_final})

        # Plot actual vs predicted BodyFat
        plt.figure(figsize=(8, 6))
        plt.scatter(test_target_hpc, pred_bodyfat_nn2_hpc_final, c='purple', marker='o')
        plt.xlabel('Actual BodyFat (HPC)')
        plt.ylabel('Predicted BodyFat (HPC)')
        plt.title(f'HPC Actual vs Predicted BodyFat\nMSE: {final_mse_hpc:.4f}, R²: {final_r2_hpc:.4f}')
        plt.xlim(min(test_target_hpc), max(test_target_hpc))
        plt.ylim(min(pred_bodyfat_nn2_hpc_final), max(pred_bodyfat_nn2_hpc_final))
        plt.axline((0, 0), slope=1, color='orange', linestyle='--')
        plt.savefig('bodyfat_pred_hpc.png')
        plt.close()
    except Exception as err:
        hpc_logger.error(f"Final model training failed: {err}")
        raise

# Release Ray resources
ray.shutdown()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 MB 7.9 MB/s eta 0:00:00


2025-04-21 20:29:03,652	INFO worker.py:1852 -- Started a local Ray instance.


+------------------------------------------------------------------------------+
| Configuration for experiment     train_bodyfat_nn2_hpc_2025-04-21_20-29-11   |
+------------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                       |
| Scheduler                        FIFOScheduler                               |
| Number of trials                 3                                           |
+------------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_bodyfat_nn2_hpc_2025-04-21_20-29-11
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-04-21_20-29-00_246868_153/artifacts/2025-04-21_20-29-11/train_bodyfat_nn2_hpc_2025-04-21_20-29-11/driver_artifacts`

Trial status: 1 PENDING
Current time: 2025-04-21 20:29:17. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+--------

2025-04-21 20:29:30,908	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_bodyfat_nn2_hpc_2025-04-21_20-29-11' in 0.0108s.



Trial train_bodyfat_nn2_hpc_488aa_00002 finished iteration 1 at 2025-04-21 20:29:30. Total running time: 13s
+------------------------------------------------------------+
| Trial train_bodyfat_nn2_hpc_488aa_00002 result             |
+------------------------------------------------------------+
| checkpoint_dir_name                                        |
| time_this_iter_s                                     1.199 |
| time_total_s                                         1.199 |
| training_iteration                                       1 |
| mse                                                0.33707 |
| r2                                                 0.99375 |
+------------------------------------------------------------+

Trial train_bodyfat_nn2_hpc_488aa_00002 completed after 1 iterations at 2025-04-21 20:29:30. Total running time: 13s

Trial status: 3 TERMINATED
Current time: 2025-04-21 20:29:30. Total running time: 13s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
Current best